In [4]:
import numpy as np
import pandas as pd

# Define the function and its derivative
def f(x):
    return np.cos(x)

# Exact derivative of cos(x) at x = pi/2
exact_derivative = -1

# Point of interest
s = np.pi / 2

# Step sizes h = 0.01 * 2^(-n), n = 0, 1, 2, ..., 9
n_values = np.arange(0, 10)
h_values = 0.01 * 2.0 ** (-n_values)

# Forward difference approximation
forward_diff = (f(s + h_values) - f(s)) / h_values

# Centered difference approximation
centered_diff = (f(s + h_values) - f(s - h_values)) / (2 * h_values)

# Errors for both methods
forward_error = np.abs(forward_diff - exact_derivative)
centered_error = np.abs(centered_diff - exact_derivative)

# Create a DataFrame to display results
results_df = pd.DataFrame({
    'h': h_values,
    'Forward Difference': forward_diff,
    'Forward Error': forward_error,
    'Centered Difference': centered_diff,
    'Centered Error': centered_error
})


results_df



,h,Forward Difference,Forward Error,Centered Difference,Centered Error
0,0.010000,-0.999983,1.666658e-05,-0.999983,1.666658e-05
1,0.005000,-0.999996,4.166661e-06,-0.999996,4.166661e-06
2,0.002500,-0.999999,1.041666e-06,-0.999999,1.041666e-06
3,0.001250,-1.000000,2.604167e-07,-1.000000,2.604167e-07
4,0.000625,-1.000000,6.510401e-08,-1.000000,6.510401e-08
5,0.000313,-1.000000,1.627624e-08,-1.000000,1.627624e-08
6,0.000156,-1.000000,4.068499e-09,-1.000000,4.068499e-09
7,0.000078,-1.000000,1.018162e-09,-1.000000,1.018162e-09
8,0.000039,-1.000000,2.552227e-10,-1.000000,2.552227e-10
9,0.000020,-1.000000,6.448786e-11,-1.000000,6.448786e-11


In [5]:
import numpy as np
import pandas as pd


# Define the vector function f(x)
def f_vector(x):
    x1, x2 = x
    return np.array([4 * x1**2 + x2**2 - 4, x1 + x2 - np.sin(x1 - x2)])

# Define the Jacobian of f(x)
def jacobian_f(x):
    x1, x2 = x
    j11 = 8 * x1  # df1/dx1
    j12 = 2 * x2  # df1/dx2
    j21 = 1 - np.cos(x1 - x2)  # df2/dx1
    j22 = 1 + np.cos(x1 - x2)  # df2/dx2
    return np.array([[j11, j12], [j21, j22]])

# Implement Slacker Newton's method
def slacker_newton(x0, epsilon=1e-10, max_iterations=100, tol_jacobian_recompute=1.2, tol_iterates=1.2):
    results = [x0]
    x = np.array(x0)
    delta = np.inf
    previous_delta = np.inf
    J_inv = np.linalg.inv(jacobian_f(x))  # Compute the initial inverse Jacobian

    for _ in range(max_iterations):
        # Evaluate the function f at the current iterate
        f_val = f_vector(x)
        
        # Perform a Newton step
        delta_x = -J_inv @ f_val
        x_new = x + delta_x
        delta = np.linalg.norm(delta_x)
        
        # Check stopping criterion
        if delta < epsilon:
            results.append(x_new)
            break
        
        # Recompute the Jacobian inverse if necessary based on conditions
        if delta > tol_jacobian_recompute * previous_delta or np.linalg.norm(x_new - x) > tol_iterates * np.linalg.norm(x - results[-1]):
            J_inv = np.linalg.inv(jacobian_f(x_new))  # Recompute the inverse of the Jacobian
        
        # Update for the next iteration
        previous_delta = delta
        x = x_new
        results.append(x)
    
    return results

# Initial guess
x0 = [1, 0]

# Perform Slacker Newton's method
slacker_newton_results = slacker_newton(x0)

# Display the results
slacker_newton_df = pd.DataFrame(slacker_newton_results, columns=['x1', 'x2'])
slacker_newton_df

,x1,x2
0,1.000000,0.000000
1,1.000000,-0.102921
2,0.998609,-0.105531
3,0.998607,-0.105530
4,0.998607,-0.105530


In [15]:
# Modify the finite difference approximation to allow different step sizes hj in each direction
def finite_difference_jacobian_variable_h(f, x, h_factors):
    n = len(x)
    J_approx = np.zeros((n, n))  # Initialize Jacobian matrix
    fx = f(x)  # Evaluate f(x) once
    
    for i in range(n):
        h = h_factors[i] * np.abs(x[i])  # Use hj = factor * |xj|
        if h == 0:  # If x[i] is zero, use a default small step size
            h = h_factors[i]
        
        # Perturb x[i] by h
        x_perturbed = np.copy(x)
        x_perturbed[i] += h
        f_perturbed = f(x_perturbed)
        
        # Compute the ith column of the Jacobian using forward difference
        J_approx[:, i] = (f_perturbed - fx) / h
        
    return J_approx

# Modify the Newton's method to accept variable step sizes for finite differences
def newton_finite_diff_variable_h(x0, epsilon=1e-10, max_iterations=100, h_factors=[1e-7, 1e-7]):
    results = [x0]
    x = np.array(x0)
    
    for _ in range(max_iterations):
        # Evaluate the function f at the current iterate
        f_val = f_vector(x)
        
        # Check stopping criterion
        if np.linalg.norm(f_val) < epsilon:
            results.append(x)
            break
        
        # Compute the approximate Jacobian using finite differences with variable step sizes
        J_approx = finite_difference_jacobian_variable_h(f_vector, x, h_factors)
        
        try:
            # Perform a Newton step by solving J * delta = -f(x)
            delta_x = np.linalg.solve(J_approx, -f_val)
        except np.linalg.LinAlgError:
            # If the matrix is singular, use the pseudo-inverse of the Jacobian
            J_pseudo_inv = np.linalg.pinv(J_approx)
            delta_x = J_pseudo_inv @ (-f_val)
        
        # Update x for the next iteration
        x_new = x + delta_x
        x = x_new
        results.append(x)
    
    return results

# Initial guess
x0 = [1, 0]

# Test case 1: hj = (10^-7)|xj|
h_factors_1 = [1e-7, 1e-7]
newton_results_h1 = newton_finite_diff_variable_h(x0, h_factors=h_factors_1)

# Test case 2: hj = (10^-3)|xj|
h_factors_2 = [1e-3, 1e-3]
newton_results_h2 = newton_finite_diff_variable_h(x0, h_factors=h_factors_2)

# Convert results to DataFrame for display
newton_results_h1_df = pd.DataFrame(newton_results_h1, columns=['x1', 'x2'])
newton_results_h1_df 




,x1,x2
0,1.000000,0.000000
1,1.000000,0.000000
2,1.000000,-0.102921
3,0.998609,-0.105531
4,0.998607,-0.105530
5,0.998607,-0.105530


In [16]:
newton_results_h2_df = pd.DataFrame(newton_results_h2, columns=['x1', 'x2'])
newton_results_h2_df

,x1,x2
0,1.000000,0.000000
1,1.000000,0.000000
2,1.000013,-0.102896
3,0.998610,-0.105530
4,0.998607,-0.105530
5,0.998607,-0.105530
6,0.998607,-0.105530


In [18]:
# Further adjust the recomputation conditions and epsilon to allow for better alignment with previous methods

def hybrid_newton_approx_jacobian_pseudo_fine_adjusted(x0, epsilon=1e-12, max_iterations=100, h_init=1e-3, tol_jacobian_recompute=1.01, tol_iterates=1.01):
    results = [x0]
    x = np.array(x0)
    delta = np.inf
    previous_delta = np.inf
    h_factors = [h_init, h_init]  # Initial hj = h_init * |xj|

    # Compute the initial approximate Jacobian
    J_inv = np.linalg.pinv(finite_difference_jacobian_variable_h(f_vector, x, h_factors))
    
    for _ in range(max_iterations):
        # Evaluate the function f at the current iterate
        f_val = f_vector(x)
        
        # Perform a Newton step
        delta_x = -J_inv @ f_val
        x_new = x + delta_x
        delta = np.linalg.norm(delta_x)
        
        # Check stopping criterion
        if delta < epsilon:
            results.append(x_new)
            break
        
        # Recompute the Jacobian inverse if necessary based on conditions
        if delta > tol_jacobian_recompute * previous_delta or np.linalg.norm(x_new - x) > tol_iterates * np.linalg.norm(x - results[-1]):
            h_factors = [h_factor / 2 for h_factor in h_factors]  # Halve the constant in front of |xj|
            J_inv = np.linalg.pinv(finite_difference_jacobian_variable_h(f_vector, x_new, h_factors))  # Recompute Jacobian pseudo-inverse
        
        # Update for the next iteration
        previous_delta = delta
        x = x_new
        results.append(x)
    
    return results

# Apply this fine-tuned method to the problem from the first set of exercises
x0 = [1, 0]

# Perform the fine-tuned hybrid root-finding method
hybrid_newton_approx_jacobian_pseudo_fine_adjusted_results = hybrid_newton_approx_jacobian_pseudo_fine_adjusted(x0)

# Display the results
hybrid_newton_approx_jacobian_pseudo_fine_adjusted_df = pd.DataFrame(hybrid_newton_approx_jacobian_pseudo_fine_adjusted_results, columns=['x1', 'x2'])
hybrid_newton_approx_jacobian_pseudo_fine_adjusted_df

,x1,x2
0,1.0,0.0
1,1.0,0.0


In [19]:
#I keep changing the thresholds but can't seem to figure out a way to keep iterating